<a href="https://colab.research.google.com/github/yogeshmotwani1/proj1/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Query in Twint API for fetching all the tweets having rblbankcares in any parmater from yesterday 8pm to today 10am**

$ twint -s rblbankcares --since "2020-08-12 20:00:00" -o file.csv --csv

In [36]:
import pandas as pd
import re
import string
import nltk
import regex
!pip install textblob
from textblob import TextBlob
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from multiprocessing import *
from multiprocessing import Pool
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv('file.csv')

In [ ]:
df.head()

In [6]:
df = df.loc[df['username']!='rblbankcares']

In [7]:
df = df.drop_duplicates(subset=['username'] ,keep="last")

In [ ]:
df['tweet']

In [9]:
def cleanTxt(text):
    text = text.lower()
    text = re.sub('@[A-Za-z0–9_]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    text = re.sub('\n','',text) # Removing newline characters
    text = re.sub('\[.*?\]', '', text) # Removing text in brackets
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Removing punctuations
    text = re.sub('\w*\d\w*', '', text) # Removing letters

 
    return text

df['tweet'] = df['tweet'].apply(cleanTxt)

In [ ]:
df['tweet'].head()

In [11]:
def remove_non_english(text):
    words = set(nltk.corpus.words.words())
    sent = text
    sent = " ".join(w for w in nltk.word_tokenize(sent) if w.lower() in words or not w.isalpha())
    return sent

df['tweet'] = df['tweet'].apply(remove_non_english)

In [13]:
def remove_hindi_tweets(text):
    mystring = text
    words = mystring.split(" ")
    for word in words:
        if regex.match('[\u0900-\u097F]+', word):
            return ""
    return mystring
df['tweet'] = df['tweet'].apply(remove_hindi_tweets)

In [14]:
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
df['tweet'] = df['tweet'].apply(lemmatize_sentence)

In [ ]:
df = df.loc[df['tweet']!='']

In [ ]:
df.reset_index().head()

In [ ]:
df['tweet'].head()

In [17]:
def vader_sentiment_scores(sentence): 
    sid_obj = SentimentIntensityAnalyzer() 
 
    sentiment_dict = sid_obj.polarity_scores(sentence) 

    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        return "Positive"
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        return "Negative" 
  
    else : 
        return "Neutral" 
  

In [18]:
df['vader_sentiment']=df.tweet.apply(vader_sentiment_scores)

In [ ]:
df['vader_sentiment']

In [20]:
def text_blob_sentiment(sentence):
    analysis = TextBlob(sentence) 
    if analysis.sentiment.polarity > 0:
        return "Postive"
    elif analysis.sentiment.polarity == 0:
        return "Neutral"
    else:
        return "Negative"

In [21]:
df['text_blob'] = df.tweet.apply(text_blob_sentiment)

In [ ]:
df['text_blob']

In [ ]:
print(len(df))
df[['tweet','vader_sentiment', 'text_blob']].to_csv('file1.csv')

In [39]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)

In [40]:
word_vectors = Word2Vec.load("../preprocessing_and_embeddings/word2vec.model").wv
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored